In [4]:
# === Evaluate saved models from main.py and visualize metrics (single notebook cell) ===
# Requirements: numpy, pandas, matplotlib, scikit-learn, joblib
import sys, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib

# If your project uses a "src" layout (data.py lives in src/), uncomment:
# sys.path.append("src")
from data import load_and_prepare_multi  # uses your existing function

# --------------------------
# Metric helpers
# --------------------------
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def wape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return 100.0 * np.sum(np.abs(y_pred - y_true)) / (np.sum(np.abs(y_true)) + eps)

def smape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    denom = np.abs(y_true) + np.abs(y_pred) + eps
    return 200.0 * np.mean(np.abs(y_pred - y_true) / denom)

def mase_denom_train(y_train_in_order, seasonality=1, eps=1e-8):
    y = np.asarray(y_train_in_order)
    if y.size <= seasonality:
        return np.nan
    d = np.abs(y[seasonality:] - y[:-seasonality])
    denom = np.mean(d) if d.size else np.nan
    if denom is None or np.isnan(denom) or denom < eps:
        denom = np.nan
    return denom

def rmse_compat(y_true, y_pred):
    # Compatible with all scikit-learn versions
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

def basic_metrics(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return {
        "MAE": mean_absolute_error(y_true, y_pred),
        "RMSE": rmse_compat(y_true, y_pred),   # <-- fixed here
        "WAPE%": wape(y_true, y_pred),
        "sMAPE%": smape(y_true, y_pred),
        "R2": r2_score(y_true, y_pred),
        "Bias": float(np.mean(y_pred - y_true)),
    }

# --------------------------
# Plot helpers (matplotlib only)
# --------------------------
def ensure_outdir(path: str | Path) -> Path:
    p = Path(path); p.mkdir(parents=True, exist_ok=True); return p

def plot_metric_by_feature(by_feature: pd.DataFrame, metric: str, out_path: Path):
    plt.figure(figsize=(8, 4))
    order = by_feature.sort_values(metric, ascending=False)
    plt.bar(order["feature"].astype(str), order[metric].values)
    plt.xticks(rotation=45, ha="right")
    plt.title(f"{metric} by Feature")
    plt.ylabel(metric)
    plt.tight_layout()
    plt.savefig(out_path, dpi=150, bbox_inches="tight"); plt.close()

def plot_top_wards(by_ward: pd.DataFrame, metric: str, topn: int, out_path: Path):
    plt.figure(figsize=(8, 6))
    order = by_ward.sort_values(metric, ascending=False).head(topn)
    plt.barh(order["ward"].astype(str), order[metric].values)
    plt.gca().invert_yaxis()
    plt.xlabel(metric); plt.title(f"Top {topn} Wards (worst {metric})")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150, bbox_inches="tight"); plt.close()

def plot_scatter_ytrue_ypred(df_test_pred: pd.DataFrame, out_path: Path):
    plt.figure(figsize=(5, 5))
    y_true = df_test_pred["y_true"].values
    y_pred = df_test_pred["y_pred"].values
    plt.scatter(y_true, y_pred, alpha=0.6, s=18)
    lims = [min(np.min(y_true), np.min(y_pred)), max(np.max(y_true), np.max(y_pred))]
    plt.plot(lims, lims)
    plt.xlim(lims); plt.ylim(lims)
    plt.xlabel("Actual"); plt.ylabel("Predicted")
    r2 = r2_score(y_true, y_pred)
    plt.title(f"y_true vs y_pred (R²={r2:.3f})")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150, bbox_inches="tight"); plt.close()

def plot_residual_hist(df_test_pred: pd.DataFrame, out_path: Path, bins=30):
    plt.figure(figsize=(7, 4))
    resid = df_test_pred["y_pred"].values - df_test_pred["y_true"].values
    plt.hist(resid, bins=bins); plt.axvline(0.0)
    plt.title("Residuals distribution (y_pred - y_true)")
    plt.xlabel("Residual"); plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(out_path, dpi=150, bbox_inches="tight"); plt.close()

# --------------------------
# Core evaluation
# --------------------------
def evaluate_and_visualize(
    EXCEL_PATH: str | Path,
    FEATURES: list[str],
    MODELS_DIR: str | Path = "models",
    OUT_DIR: str | Path = "reports/figs_eval",
    id_col: str = "id",
    year_col: str = "year",
    seasonality: int = 1,     # yearly 1-step ahead
):
    MODELS_DIR = Path(MODELS_DIR)
    OUT_DIR = ensure_outdir(OUT_DIR)

    # Registry + encoder saved by main.py
    registry = json.loads(Path(MODELS_DIR, "registry.json").read_text(encoding="utf-8"))
    encoder = joblib.load(MODELS_DIR / "encoder.pkl")
    n_ids = len(encoder.categories_[0])

    # Infer the sequence_length used in training from any scaler (n_features = k + n_ids)
    any_feat = next(iter(registry.keys()))
    k = joblib.load(registry[any_feat]["scaler"]).mean_.shape[0] - n_ids
    assert k > 0, "Invalid inferred sequence length."
    print(f"Detected sequence_length used in training: k = {k}")

    # Build windows with the SAME k
    X_map, Y_map, META_map, _ = load_and_prepare_multi(
        filepath=str(EXCEL_PATH),
        feature_cols=FEATURES,
        sequence_length=k,
        id_col=id_col,
        year_col=year_col,
    )

    # Collect per-sample predictions on the test split (same split rule as main.py)
    rows_test, rows_train_for_mase = [], []
    for feat, info in registry.items():
        if feat not in X_map:
            continue  # was skipped or not requested

        last_n_years_test = info.get("last_n_years_test", 2)
        X, y, meta = X_map[feat], Y_map[feat], META_map[feat].copy().reset_index(drop=True)

        # Build test mask: last N target years per id
        test_mask = np.zeros(len(meta), dtype=bool)
        for _id, grp in meta.groupby(id_col):
            years_sorted = np.sort(grp["target_year"].unique())
            last_n = years_sorted[-last_n_years_test:] if len(years_sorted) >= last_n_years_test else years_sorted
            mask = grp["target_year"].isin(last_n).values
            test_mask[grp.index] = mask

        X_train, X_test = X[~test_mask], X[test_mask]
        y_train, y_test = y[~test_mask], y[test_mask]
        meta_train = meta.loc[~test_mask, :]
        meta_test  = meta.loc[test_mask, :]

        # Load scalers + model
        x_scaler = joblib.load(info["scaler"])
        y_scaler = joblib.load(info["y_scaler"]) if info.get("y_scaler") else None
        model = joblib.load(info["model"])

        # Transform + predict
        X_train_s = x_scaler.transform(X_train)
        X_test_s  = x_scaler.transform(X_test)
        yhat_train_s = model.predict(X_train_s)
        yhat_test_s  = model.predict(X_test_s)

        # Inverse y scaling where applicable
        if y_scaler is not None:
            yhat_train = y_scaler.inverse_transform(yhat_train_s.reshape(-1,1)).ravel()
            yhat_test  = y_scaler.inverse_transform(yhat_test_s.reshape(-1,1)).ravel()
        else:
            yhat_train, yhat_test = yhat_train_s, yhat_test_s

        # Accumulate test rows
        rows_test.extend([
            {
                "feature": feat,
                "ward": meta_test.iloc[i][id_col],
                "year": int(meta_test.iloc[i]["target_year"]),
                "y_true": float(y_test[i]),
                "y_pred": float(yhat_test[i]),
            }
            for i in range(len(y_test))
        ])

        # Accumulate TRAIN y values per (feature, ward) for MASE denominators
        rows_train_for_mase.extend([
            {
                "feature": feat,
                "ward": meta_train.iloc[i][id_col],
                "year": int(meta_train.iloc[i]["target_year"]),
                "y": float(y_train[i]),
            }
            for i in range(len(y_train))
        ])

    df_test_pred = pd.DataFrame(rows_test)
    df_train = pd.DataFrame(rows_train_for_mase)

    # ---- Metrics: overall / by-feature / by-ward / per(feature,ward)
    overall = pd.Series(basic_metrics(df_test_pred["y_true"], df_test_pred["y_pred"]), name="overall")

    by_feature = (
        df_test_pred
        .groupby("feature")
        .apply(lambda g: pd.Series(basic_metrics(g["y_true"], g["y_pred"])))
        .reset_index()
    )

    by_ward = (
        df_test_pred
        .groupby("ward")
        .apply(lambda g: pd.Series(basic_metrics(g["y_true"], g["y_pred"])))
        .reset_index()
    )

    per_pair = (
        df_test_pred
        .groupby(["feature","ward"])
        .apply(lambda g: pd.Series(basic_metrics(g["y_true"], g["y_pred"])))
        .reset_index()
    )

    # Add MASE per (feature, ward) using TRAIN series only
    denoms = (
        df_train
        .sort_values(["feature","ward","year"])
        .groupby(["feature","ward"])["y"]
        .apply(lambda s: mase_denom_train(s.values, seasonality=seasonality))
        .rename("MASE_denom")
        .reset_index()
    )
    per_pair = per_pair.merge(denoms, on=["feature","ward"], how="left")
    per_pair["MASE"] = per_pair["MAE"] / per_pair["MASE_denom"]

    # ---- Save tables
    OUT_DIR = ensure_outdir(OUT_DIR)
    overall.to_csv(OUT_DIR / "metrics_overall.csv")
    by_feature.to_csv(OUT_DIR / "metrics_by_feature.csv", index=False)
    by_ward.to_csv(OUT_DIR / "metrics_by_ward.csv", index=False)
    per_pair.to_csv(OUT_DIR / "metrics_by_feature_ward.csv", index=False)

    # ---- Plots
    plot_metric_by_feature(by_feature, metric="sMAPE%", out_path=OUT_DIR / "smape_by_feature.png")
    plot_metric_by_feature(by_feature, metric="WAPE%", out_path=OUT_DIR / "wape_by_feature.png")
    plot_top_wards(by_ward, metric="WAPE%", topn=15, out_path=OUT_DIR / "worst_wards_wape.png")
    plot_scatter_ytrue_ypred(df_test_pred, out_path=OUT_DIR / "scatter_true_vs_pred.png")
    plot_residual_hist(df_test_pred, out_path=OUT_DIR / "residuals_hist.png")

    print(f"Saved tables and figures to: {OUT_DIR.resolve()}")
    return {
        "overall": overall,
        "by_feature": by_feature,
        "by_ward": by_ward,
        "per_pair": per_pair,
        "df_test_pred": df_test_pred,
        "df_train": df_train,
        "sequence_length_detected": k,
    }

# --------------------------
# === Example input form / usage ===
EXCEL_PATH = "../new_data/ward_emission_cleaned.xlsx"   # <-- adjust if needed
FEATURES = [
    "area",
    "riceEmissionCH4", "riceEmissionN2O",
    "livestockEmissionCH4", "livestockEmissionN2O",
    "carbonSequestrationBelowGround", "carbonSequestrationAboveGround",
    "CH4Emission", "N2OEmission",
]
MODELS_DIR = "models"
OUT_DIR = "reports/figs_eval"

# 4) Run evaluation
results = evaluate_and_visualize(EXCEL_PATH, FEATURES, MODELS_DIR, OUT_DIR)

# Optional quick peeks:
# results["overall"], results["by_feature"].head(), results["by_ward"].head(), results["per_pair"].head()


Detected sequence_length used in training: k = 4


/tmp/ipykernel_4077358/1864586195.py:208: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(basic_metrics(g["y_true"], g["y_pred"])))
/tmp/ipykernel_4077358/1864586195.py:215: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series(basic_metrics(g["y_true"], g["y_pred"])))
/mnt/disk1/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_regression.py:1266: Undefine

Saved tables and figures to: /mnt/disk3/anhnd2468/CarbonFarmingCode/src/reports/figs_eval
